In [15]:
import xlrd
import os
import random
import jieba
import pickle
import csv
from collections import Counter

In [16]:
workbook = xlrd.open_workbook(r'./data/taxdata3_3.xlsx')

# 获取第1张表
sheet1 = workbook.sheet_by_index(0)
sheet1_cols0 = sheet1.col_values(0) 
sheet1_cols1 = sheet1.col_values(1) 
sheet1_cols2 = sheet1.col_values(2) 
sheet1_cols3 = sheet1.col_values(3) 
sheet1_class_list = sheet1.col_values(4)
sheet1_class_list = sheet1_class_list[1:]
sheet1_col_length = len(sheet1_cols0)
raw_data1_list = []
for i in range(1, sheet1_col_length):
    k = sheet1_cols0[i] + sheet1_cols1[i] + sheet1_cols2[i] + sheet1_cols3[i]
    raw_data1_list.append(k.replace("\n",""))

# 获取第2张表
sheet2 = workbook.sheet_by_index(1)
sheet2_cols0 = sheet2.col_values(0) 
sheet2_cols1 = sheet2.col_values(1) 
sheet2_cols2 = sheet2.col_values(2) 
sheet2_cols3 = sheet2.col_values(3) 
sheet2_class_list = sheet2.col_values(4)
sheet2_class_list = sheet2_class_list[1:]
sheet2_col_length = len(sheet2_cols0)
raw_data2_list = []
for i in range(1, sheet2_col_length):
    k = sheet2_cols0[i] + sheet2_cols1[i] + sheet2_cols2[i] + sheet2_cols3[i]
    raw_data2_list.append(k.replace("\n",""))

# 获取第3张表
sheet3 = workbook.sheet_by_index(2)
sheet3_cols0 = sheet3.col_values(0) 
sheet3_cols1 = sheet3.col_values(1) 
sheet3_cols2 = sheet3.col_values(2) 
sheet3_cols3 = sheet3.col_values(3) 
sheet3_class_list = sheet3.col_values(4)
sheet3_class_list = sheet3_class_list[1:]
sheet3_col_length = len(sheet3_cols0)
raw_data3_list = []
for i in range(1, sheet3_col_length):
    k = sheet3_cols0[i] + sheet3_cols1[i] + sheet3_cols2[i] + sheet3_cols3[i]
    raw_data3_list.append(k.replace("\n",""))

#data 集合
raw_data_list = []
raw_data_list.extend(raw_data1_list)
raw_data_list.extend(raw_data2_list)
raw_data_list.extend(raw_data3_list)

#class 集合
class_list = []
class_list.extend(sheet1_class_list)
class_list.extend(sheet2_class_list)
class_list.extend(sheet3_class_list)

In [17]:
'''
综合税种=综合，两个类的、所得税=综合，发票管理、发票管理管理=发票，财务类=企业会计准则，财务类，
个税=个人所得税，环境保护税=环保税，土地使用税=城镇土地使用税，
教育费附加、城建税、教育费附加=附加税费，
出口退税=进出口税收，税收风险=税务风险
'''
for i in range(len(class_list)):
    if class_list[i] == '' or class_list[i] == '工业企业':
        class_list[i] = "未分类"
    if class_list[i]=='综合税种' or class_list[i]=='多税种' or class_list[i]=='所得税' or (' ' in class_list[i]) or ('，' in class_list[i]) or ('、' in class_list[i]) or ('\u3000' in class_list[i]):
        class_list[i] = "综合"
    class_list[i] = class_list[i].replace(' ', '')
    class_list[i] = class_list[i].replace('\n', '')
    if class_list[i] == '发票管理' or class_list[i] == '发票管理管理':
        class_list[i] = "发票"
    if class_list[i] == '企业会计准则，财务类':
        class_list[i] = "财务类"
    if class_list[i] == '土地使用税':
        class_list[i] = "城镇土地使用税"
    if class_list[i] == '附加税' or class_list[i] == '教育费附加' or class_list[i] == '城建税':
        class_list[i] = "附加税费"
    if class_list[i] == '出口退税':
        class_list[i] = "进出口税收"   
    if class_list[i] == '税收风险':
        class_list[i] = "税务风险"
    if class_list[i] == '印花':
        class_list[i] = "印花税"
    if class_list[i] == '纳税社保' or class_list[i] == '社保' or class_list[i] == '个税' or class_list[i] == '个人所得说' or class_list[i] == '新个税':
        class_list[i] = "个人所得税"   
    if class_list[i] == '营改增' or class_list[i] == '增值况':
        class_list[i] = "增值税"    
    if class_list[i] == '环保税':
        class_list[i] = "环境保护税" 
    if class_list[i] == '企业所得呼'or class_list[i] == '企业所得税法':
        class_list[i] = "企业所得税" 
    if class_list[i] == '土增税':
        class_list[i] = "土地增值税"
    if class_list[i] == '营改增':
        class_list[i] = "增值税"
    if class_list[i] == '税收优惠' or class_list[i] == '税务登记' or class_list[i] == '税务风险' or class_list[i] == '税务变更' or class_list[i] == '税务注销' or class_list[i] == '税务征管':
        class_list[i] = "税收征管"
    if class_list[i] == '申报征收':
        class_list[i] = '纳税申报'
    if class_list[i] == '关税':
        class_list[i] = "涉外税收"
        
c = Counter(class_list)
topn_class = 4       #取最多class的类别
data_number = 1001     #取data数目
class_type_list = [k[0] for k in c.most_common(topn_class)]
class_type_len = len(list(c.most_common(topn_class)))
print("class type total:", class_type_len)

data_class_list = list(zip(raw_data_list, class_list))  # zip压缩合并，将数据与标签对应压缩
random.shuffle(data_class_list)  # 将data_class_list乱序 
raw_data_list, class_list = zip(*data_class_list)  


#取最多类别的前n个样本
new_class_list = []
new_data_list = []
#todo: 每个类别取k个样本试试看
count_n = [1 for i in range(class_type_len)]        
for i in range(class_type_len):
    for j in range(len(class_list)):
        if class_list[j] == class_type_list[i] and count_n[i] < data_number:
            new_class_list.append(class_list[j])
            new_data_list.append(raw_data_list[j])
            count_n[i] += 1

class type total: 4


In [18]:
c.most_common(14)

[('企业所得税', 2502),
 ('增值税', 2428),
 ('个人所得税', 1611),
 ('综合', 1055),
 ('发票', 854),
 ('税收征管', 759),
 ('财务类', 621),
 ('印花税', 418),
 ('房产税', 309),
 ('进出口税收', 294),
 ('消费税', 239),
 ('契税', 214),
 ('未分类', 213),
 ('土地增值税', 210)]

In [19]:
print(new_class_list[199:201])
print(new_data_list[199:201])

['企业所得税', '企业所得税']
['已购买职业责任保险的税务师事务所如何计算抵扣保险受益年度的应提职业风险基金金额？\u3000\u3000根据《国家税务总局关于发布〈税务师事务所职业风险基金管理办法〉的公告》(国家税务总局公告2010年第14号)第六条规定：税务师事务所可以通过购买职业责任保险方式提高抵御职业责任风险的能力。    税务师事务所购买职业保险的，实际缴纳的保险费可以按以下公式计算抵扣保险受益年度的应提职业风险基金金额：可抵扣金额=本年度负担的保险费*5。    可抵扣金额大于或等于本年度应提职业风险基金金额的，本年度可以不再提取职业风险基金。    可抵扣金额小于本年度应提取职业风险基金金额的，应当按其差额提取职业风险基金。    税务师事务所以保险费抵扣应提职业风险基金金额的，应当于签订保险合同后1个月之内，将保单（含保险条款）复印件报所在地省注册税务师管理中心、省注册税务师协会备案。\u3000\u3000    第十五条规定：本办法自2010年10月1日起实施。', '企业销售房地产开发产品的收入确认时间是如何规定的？    根据《国家税务总局关于印发<房地产开发经营业务企业所得税处理办法>的通知》（国税发〔2009〕31号）第六条规定：“企业通过正式签订《房地产销售合同》或《房地产预售合同》所取得的收入，应确认为销售收入的实现，具体按以下规定确认：\u3000\u3000（一）采取一次性全额收款方式销售开发产品的，应于实际收讫价款或取得索取价款凭据（权利）之日，确认收入的实现。\u3000\u3000（二）采取分期收款方式销售开发产品的，应按销售合同或协议约定的价款和付款日确认收入的实现。付款方提前付款的，在实际付款日确认收入的实现。\u3000\u3000（三）采取银行按揭方式销售开发产品的，应按销售合同或协议约定的价款确定收入额，其首付款应于实际收到日确认收入的实现，余款在银行按揭贷款办理转账之日确认收入的实现。\u3000\u3000（四）采取委托方式销售开发产品的，应按以下原则确认收入的实现：    1.采取支付手续费方式委托销售开发产品的，应按销售合同或协议中约定的价款于收到受托方已销开发产品清单之日确认收入的实现。\u3000\u30002.采取视同买断方式委托销售开发产品的，属于企业与购买方签订销售合同或协议，或企业、受托

In [20]:
#分词
new_data_list_cut = []
rowlength = len(new_data_list)
for i in range(rowlength):
    word_cut = jieba.cut(new_data_list[i], cut_all=False)
    word_list = list(word_cut) 
    new_data_list_cut.append(word_list)


Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/wn/7md2z8rj1td41mc56_6lr3jr0000gn/T/jieba.cache
Loading model cost 0.944 seconds.
Prefix dict has been built succesfully.


In [21]:
def MakeWordsSet(words_file):
    words_set = set()  
    with open(words_file, 'r', encoding='utf-8') as f:  
        for line in f.readlines():  
            word = line.strip()  
            if len(word) > 0:  
                words_set.add(word)
    return words_set  

In [22]:
def words_dict(new_data_list, stopwords_set=set()):
    data_featrue_list = []  
    for t in range(1, len(new_data_list)):
        # 如果不是指定的结束语，并且单词长度大于1小于5，作为特征词
        if new_data_list[t] not in stopwords_set and 1 < len(new_data_list[t]) < 5:
            data_featrue_list.append(new_data_list[t])
    return data_featrue_list

In [23]:
# 生成stopwords_set
stopwords_file = './data/stopwords_cn.txt'
stopwords_set = MakeWordsSet(stopwords_file)


In [24]:
data_featrue_list = []
for i in range(len(new_data_list_cut)):
    data_featrue = words_dict(new_data_list_cut[i], stopwords_set)
    data_featrue_list.append(data_featrue)

In [25]:
# print(new_class_list[199:201])
# print(data_featrue_list[199:201])

In [26]:
cc = Counter(new_class_list)
cc.most_common()

[('企业所得税', 1000), ('增值税', 1000), ('个人所得税', 1000), ('综合', 1000)]

In [27]:
new_class_list[9]

'企业所得税'

In [28]:
data_class_list = list(zip(data_featrue_list, new_class_list))  # zip压缩合并，将数据与标签对应压缩
random.shuffle(data_class_list)  # 将data_class_list乱序
index = int(len(data_class_list) * 0.25) + 1  # 训练集和测试集切分的索引值
train_list = data_class_list[index:]  # 训练集
test_list = data_class_list[:index]  # 测试集
train_data_list, train_class_list = zip(*train_list)  # 训练集解压缩
test_data_list, test_class_list = zip(*test_list)  # 测试集解压缩

In [29]:
print(len(data_featrue_list))
print(len(new_class_list))

4000
4000


In [30]:
print([data_featrue_list[i], new_class_list[4]])

[['跨期', '发票', '费用', '处理', '问题', '实务', '存在', '大量', '业务', '发生', '12', '月底', '时间', '地域', '造成', '发票', '无法', '传递', '具体', '金额', '费用', '无法', '预知', '费用', '票据', '是否', '年度', '列支', '所得税', '扣除', '企业', '所得税法', '实施', '条例', '第九条', '规定', '企业', '纳税', '所得额', '计算', '权责', '发生', '原则', '属于', '当期', '收入', '费用', '款项', '是否', '收付', '当期', '收入', '费用', '属于', '当期', '收入', '费用', '款项', '已经', '当期', '收付', '当期', '收入', '费用', '企业', '所得税', '纳税', '所得额', '若干', '税务', '处理', '问题', '公告', '公告', '2012', '15', '第六条', '规定', '税收', '征收', '管理法', '规定', '企业', '发现', '以前年度', '实际', '发生', '税收', '规定', '企业', '所得税', '扣除', '扣除', '扣除', '支出', '企业', '做出', '专项', '申报', '说明', '准予', '追补', '至该', '项目', '发生', '年度', '计算', '扣除', '追补', '确认', '期限', '不得', '超过', '根据上述', '规定', '贵司', '2017', '发生', '费用', '需要', '2017', '企业', '所得税', '税前', '扣除', '权责', '发生', '原则', '应当', '发票', '税前', '扣除', '成本费用', '项目', '企业', '当年', '没有', '取得', '发票', '次年', '企业', '所得税', '汇算', '清缴', '期限', '取得', '发票', '该项', '成本费用', '发生', '年度', '税前', '扣除', '无需', '纳税', '调整'], '企业所得税']


In [31]:
train_data = []
# train_data.append(["content", "label"])
for i in range(len(train_data_list)):
    k = ''.join(train_data_list[i])
    train_data.append([k, train_class_list[i]])
print(train_data[0])

with open('./data/train_taxdata_0107_stopwords.csv', "w", newline='') as fw:
    writer = csv.writer(fw)
    for row in train_data:
        writer.writerow(row)
    fw.close()
    
train_text = []
with open("./data/train_taxdata_0107_stopwords.csv",'r') as f:
    reader = csv.reader(f)
    for item in reader:
        train_text.append(list(item))
fw.close()
print(train_text[1])

['奖励财务人员返还是否缴税财政部若干政策问题通知1994财税字20文件规定办理代扣代缴税款手续按规定取得扣缴手续费暂免征收规定返还手续费发给办税人员办税人员享受免征若此财务人员办税相关人员文件认为需要就此返还收入缴纳', '个人所得税']
['开发经营企业土地成本原则分配成本对象印发房地产开发经营业务企业所得税处理办法通知国税发200931第三十条规定企业下列成本以下方法进行分配土地成本一般占地面积进行分配确需结合方法进行分配应商税务机关同意土地开发连结房地产开发属于一次性取得土地分期开发房地产情况土地开发成本经商税务机关同意可先土地整体预算成本进行分配土地整体开发完毕再行调整国税发200931印发房地产开发经营业务企业所得税处理办法通知第三十条', '企业所得税']


In [33]:
dev_data = []
# dev_data.append(["content", "label"])
for i in range(len(test_data_list)):
    k = ''.join(test_data_list[i])
    dev_data.append([k, test_class_list[i]])
print(dev_data[0])

with open('./data/dev_taxdata_0107_stopwords.csv', "w", newline='') as fw:
    writer = csv.writer(fw)
    for row in dev_data:
        writer.writerow(row)
    fw.close()
    
dev_text = []
with open("./data/dev_taxdata_0107_stopwords.csv",'r') as f:
    reader = csv.reader(f)
    for item in reader:
        dev_text.append(list(item))
fw.close()
print(dev_text[1])


['请教支付国外佣金纳税支付国外佣金视为贵司提供服务收取服务费属于非贸付范畴以下情况分析提供劳务发生地全境贵司对外支付需要代扣代缴6%税率增值税增值税附加提供劳务发生地境内贵司对外支付需要代扣代缴6%税率增值税增值税附加劳务报酬所得代扣代缴', '综合']
['实施国家重点扶持公共基础设施项目企业所得税优惠问题通知中所称内部自用实施国家重点扶持公共基础设施项目企业所得税优惠问题通知国税发200980规定企业从事承包经营承包建设内部自用目录规定项目所得不得享受前款规定企业所得税优惠通知内部自用项目建设企业主体经营业务设施满足企业生产经营活动需要属于提供公共服务业务公共基础设施建设项目国税发200980实施国家重点扶持公共基础设施项目企业所得税优惠问题通知', '企业所得税']


In [5]:
import tensorflow as tf
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


In [8]:
from tensorflow.python import pywrap_tensorflow

ModuleNotFoundError: No module named 'swig'

In [6]:
print(tf.__file__)

/Users/xiezhijun/anaconda3/lib/python3.6/site-packages/tensorflow/__init__.py


In [103]:
from bert_serving.client import BertClient
bc = BertClient()
embedding = bc.encode(['somebody', 'book', 'sunday'])
print(embedding)
print(len(embedding[1]))

[[ 0.08143956  0.24072137  0.17423196 ... -0.20935664 -0.00765664
   0.00987268]
 [-0.32008436  0.6250418  -0.46648058 ...  0.09882972  0.02564905
  -0.17454903]
 [-0.10663307  0.77239317 -0.2586998  ...  0.10752412  0.06072805
  -0.6352674 ]]
768


In [92]:
import sys
print(sys.version)
print(sys.version_info)
print(sys.path)

3.6.7 |Anaconda custom (64-bit)| (default, Oct 23 2018, 14:01:38) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
['', '/Users/xiezhijun/anaconda3/lib/python36.zip', '/Users/xiezhijun/anaconda3/lib/python3.6', '/Users/xiezhijun/anaconda3/lib/python3.6/lib-dynload', '/Users/xiezhijun/anaconda3/lib/python3.6/site-packages', '/Users/xiezhijun/anaconda3/lib/python3.6/site-packages/aeosa', '/Users/xiezhijun/anaconda3/lib/python3.6/site-packages/xgboost-0.7-py3.6.egg', '/Users/xiezhijun/anaconda3/lib/python3.6/site-packages/IPython/extensions', '/Users/xiezhijun/.ipython']


In [97]:
import sys
sys.executable

'/Users/xiezhijun/anaconda3/bin/python'

In [11]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import csv
import os
import modeling
import optimization
import tokenization
import tensorflow as tf
import platform
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import pickle

In [13]:
flags = tf.flags

FLAGS = flags.FLAGS
flags.DEFINE_string(
    "data_dir", None,
    "The input data dir. Should contain the .tsv files (or other data files) "
    "for the task.")

In [14]:
flags.mark_flag_as_required("data_dir")